In [155]:
import pandas as pd

# Bonus 1

In [156]:
df = pd.read_csv('../data/raw/full_data.csv')

In [157]:
df.rename(columns={'dem_education_level': 'Education Level',
                   'question_bbi_2016wave4_basicincome_argumentsfor': 'Pro Arguments',
                   'question_bbi_2016wave4_basicincome_argumentsagainst': 'Cons Arguments',
                  'question_bbi_2016wave4_basicincome_vote': 'Position',
                  'question_bbi_2016wave4_basicincome_effect': 'Effect',
                  'question_bbi_2016wave4_basicincome_awareness': 'Awareness'}, inplace=True)

In [158]:
def split(x):
    return x.split('|')

In [159]:
df['Pro Arguments'] = df.apply(lambda x: split(x['Pro Arguments']), axis=1)

In [160]:
df['Cons Arguments'] = df.apply(lambda x: split(x['Cons Arguments']), axis=1)

In [161]:
def ind_range(x):
    if x == ['None of the above']:
        return 0
    else:
        return len(x)

In [162]:
df['Pro Arguments'] = df.apply(lambda x: ind_range(x['Pro Arguments']), axis=1)

In [163]:
df['Cons Arguments'] = df.apply(lambda x: ind_range(x['Cons Arguments']), axis=1)

In [146]:
df['Position'].unique()

array(['I would not vote', 'I would probably vote for it',
       'I would vote against it', 'I would vote for it',
       'I would probably vote against it'], dtype=object)

In [165]:
def position(x):
    if 'vote for' in x:
        return 'In Favor'
    elif 'vote against' in x:
        return 'Against'
    else:
        return 'Impartial'

In [166]:
df['Position'] = df.apply(lambda x: position(x['Position']), axis=1)

In [172]:
bonus1 = df.groupby('Position')['Pro Arguments', 'Cons Arguments'].sum()

<ipython-input-172-b2d51c156da2>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bonus1 = df.groupby('Position')['Pro Arguments', 'Cons Arguments'].sum()


In [176]:
bonus1 = bonus1.rename(columns={'Pro Arguments':'Number of Pro Arguments', 'Cons Arguments':'Number of Cons Arguments'}).drop(index='Impartial')

In [177]:
bonus1

,Number of Pro Arguments,Number of Cons Arguments
Position,,
Against,2195,6558
In Favor,11753,10928


# Bonus 2

In [181]:
df['Education Level'] = df['Education Level'].replace('no', 'no education')

In [189]:
df['Education Level'].describe(include='all')

count       8986
unique         4
top       medium
freq        3578
Name: Education Level, dtype: object